In [0]:
%run /Workspace/Users/aminah.yussuf@slalom.com/Includes/Copy-Datasets

In [0]:
%sql
CREATE TABLE IF NOT EXISTS books_csv
  (book_id STRING, title STRING, author STRING, category STRING, price DOUBLE)
USING CSV
OPTIONS ( 
  header = "true", 
  delimiter = ";"
)
LOCATION "dbfs:/mnt/demo-datasets/bookstore/books-csv";


select * from books_csv

In [0]:
(spark.readStream
 .table("books_csv")
 .createOrReplaceTempView("books_streaming_tmp_vw")
)



In [0]:
%sql
select * from books_streaming_tmp_vw

In [0]:
%sql
SELECT author, count(book_id) AS total_books 
FROM books_streaming_tmp_vw
GROUP BY author

In [0]:
%sql
----create a new streaming temporary view 
CREATE OR REPLACE TEMP VIEW author_counts_tmp_vw AS (
  SELECT author, count(book_id) AS total_books
  FROM books_streaming_tmp_vw
  GROUP BY author
)


In [0]:
#use spark table to load a streaming temporary view into a dataframe. Spark always load streaming views as a streaming dataframe and static views as static dataframe. Here we use dataframe wrriteStream method to persist the result of a streaming query to a durable storage
(spark.table("author_counts_tmp_vw")
    .writeStream
    .trigger(processingTime='4 seconds')\
    .outputMode("complete")
    .option("checkpointLocation", "dbfs:/mnt/demo-datasets/author_counts_tmp_vw_checkpoint")
    .table("author_counts")
)
 

In [0]:
%sql
select * from author_counts;

In [0]:
#a triggered incremental batch, in this case the query will process all new available data and stop when it is done. we use awaitTermination=true method to block the execution of any cell in this notebook until the incremental batch write succeeds. 
(spark.table("author_counts_tmp_vw")
    .writeStream
    .trigger(availableNow=True)
    .outputMode("complete")
    .option("checkpointLocation", "dbfs:/mnt/demo-datasets/author_counts_tmp_vw_checkpoint")
    .table("author_counts")
    .awaitTermination()
)

In [0]:
%sql
select * from author_counts